# Chatfield Graph Visualization Examples

This notebook demonstrates how to visualize the LangGraph workflows that power Chatfield's Socratic dialogues.

## Setup

First, make sure you have the visualization dependencies installed:

```bash
pip install chatfield[notebook]
# or
pip install chatfield[visualization]
```

In [ ]:
# Import required libraries
import os
from IPython.display import Image, display
import chatfield
from chatfield import (
    gather, must, reject, hint, user, agent,
    get_agent_graph, get_graph_from_class, 
    visualize_graph, display_graph_in_notebook,
    process_socrates_class
)

# Set your OpenAI API key if not already set
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

print(f"Chatfield version: {chatfield.__version__}")

## Example 1: Simple Form Graph

Let's start with a simple form that collects basic information:

In [ ]:
@gather
class SimpleForm:
    """A simple contact form."""
    
    def name():
        """What is your name?"""
        pass
    
    def email():
        """What is your email address?"""
        pass
    
    def message():
        """What message would you like to send?"""
        pass

In [ ]:
# Extract the metadata and create the graph
simple_meta = process_socrates_class(SimpleForm)
simple_graph = get_agent_graph(simple_meta)

# Visualize the graph
print("Simple Form Workflow:")
display(Image(simple_graph.get_graph().draw_mermaid_png()))

## Example 2: Form with Validation Rules

Now let's create a more complex form with validation rules:

In [ ]:
@gather
@user("A startup founder seeking investment")
@agent("Be encouraging but thorough in gathering details")
class StartupPitchForm:
    """Help me understand your startup idea so I can help you craft a compelling pitch."""
    
    @hint("What problem are you solving?")
    @must("specific problem")
    @must("target audience")
    def problem():
        """Describe the problem your startup solves"""
        pass
    
    @hint("How does your product work?")
    @must("clear explanation")
    @reject("vague descriptions")
    def solution():
        """Explain your solution"""
        pass
    
    @hint("What makes you different?")
    @must("competitive advantage")
    def differentiation():
        """What makes your solution unique?"""
        pass
    
    @hint("How will you make money?")
    @must("revenue model")
    @reject("we'll figure it out later")
    def business_model():
        """Describe your business model"""
        pass

In [ ]:
# Create and visualize the graph with validation
startup_graph = get_graph_from_class(StartupPitchForm)

print("Startup Pitch Form Workflow (with validation):")
display(Image(startup_graph.get_graph().draw_mermaid_png()))

## Example 3: Using the Convenience Functions

Chatfield provides several convenience functions for working with graphs:

In [ ]:
@gather
class CustomerFeedback:
    """Help us improve our service with your feedback."""
    
    @hint("What did you use our service for?")
    def use_case():
        """What brought you to our service today?"""
        pass
    
    @must("specific feedback")
    @hint("Be as detailed as possible")
    def experience():
        """How was your experience?"""
        pass
    
    def suggestions():
        """Any suggestions for improvement?"""
        pass

In [ ]:
# Method 1: Direct visualization in notebook
feedback_graph = get_graph_from_class(CustomerFeedback)
print("Method 1: Using display_graph_in_notebook()")
display_graph_in_notebook(feedback_graph)

In [ ]:
# Method 2: Save to file
print("Method 2: Saving graph to file")
visualize_graph(feedback_graph, "customer_feedback_workflow.png")
print("Graph saved!")

In [ ]:
# Method 3: Get graph metadata
from chatfield import get_graph_metadata

metadata = get_graph_metadata(feedback_graph)
print("Graph Metadata:")
print(f"  Nodes: {metadata.get('nodes', [])}")
print(f"  Number of nodes: {metadata.get('node_count', 0)}")
print(f"  Number of edges: {metadata.get('edge_count', 0)}")

## Example 4: Complex Workflow with Multiple Validation Rules

Let's create a more complex example that shows how the graph changes with different validation rules:

In [ ]:
@gather
@user("Someone reporting a technical issue")
@agent("Be patient and thorough in gathering diagnostic information")
class BugReport:
    """Help me understand the issue you're experiencing so we can fix it quickly."""
    
    @must("specific description")
    @must("when it occurs")
    @reject("it doesn't work")
    @hint("What exactly happens? When does it occur?")
    def issue_description():
        """Describe the issue you're experiencing"""
        pass
    
    @must("steps to reproduce")
    @hint("Walk me through what you did before the issue occurred")
    def reproduction_steps():
        """How can we reproduce this issue?"""
        pass
    
    @must("expected behavior")
    @must("actual behavior")  
    def expected_vs_actual():
        """What did you expect to happen vs what actually happened?"""
        pass
    
    @hint("Browser, OS, app version, etc.")
    def environment():
        """What system/environment are you using?"""
        pass
    
    def additional_info():
        """Any other relevant information?"""
        pass

In [ ]:
# Visualize the complex workflow
bug_meta = process_socrates_class(BugReport)
bug_graph = get_agent_graph(bug_meta, max_retries=5)  # Allow more retries for complex validation

print("Bug Report Workflow (complex validation):")
display(Image(bug_graph.get_graph().draw_mermaid_png()))

# Show field information
print("\nField Information:")
for field_name, field in bug_meta.fields.items():
    print(f"\n{field_name}:")
    print(f"  Description: {field.description}")
    if field.must_rules:
        print(f"  Must include: {', '.join(field.must_rules)}")
    if field.reject_rules:
        print(f"  Must not include: {', '.join(field.reject_rules)}")
    if field.hints:
        print(f"  Hints: {', '.join(field.hints)}")

## Example 5: Comparing Different Graph Configurations

Let's see how different configurations affect the graph structure:

In [ ]:
# Create graphs with different retry settings
@gather
class TestForm:
    @must("valid email format")
    def email():
        """Your email address"""
        pass
    
    @must("at least 8 characters")
    def password():
        """Choose a password"""
        pass

test_meta = process_socrates_class(TestForm)

# Create graphs with different configurations
graph_default = get_agent_graph(test_meta, max_retries=3)
graph_strict = get_agent_graph(test_meta, max_retries=1)
graph_lenient = get_agent_graph(test_meta, max_retries=10)

print("Graph with default retries (3):")
display(Image(graph_default.get_graph().draw_mermaid_png()))

## Example 6: Accessing Graph Internals

For advanced use cases, you can access the graph structure directly:

In [ ]:
from chatfield import ChatfieldAgent

@gather  
class AdvancedForm:
    def field1():
        """First field"""
        pass
    
    def field2():
        """Second field"""
        pass

# Create agent and access its graph
advanced_meta = process_socrates_class(AdvancedForm)
agent = ChatfieldAgent(advanced_meta)

# Get the graph and its structure
graph = agent.get_graph()
structure = agent.get_graph_structure()

print("Graph Structure Details:")
print(f"Nodes: {structure['nodes']}")
print(f"\nEdges: {structure['edges']}")
print(f"\nConditional edges: {structure['conditional_edges']}")
print(f"\nEntry point: {structure['entry_point']}")
print(f"\nMetadata: {structure['meta']}")

# Visualize
print("\nGraph Visualization:")
display(Image(graph.get_graph().draw_mermaid_png()))

## Tips for Using Graph Visualization

1. **Installation**: Make sure you have the visualization dependencies:
   ```bash
   pip install chatfield[visualization]
   ```

2. **Large Graphs**: For forms with many fields, the graph can become complex. Consider breaking down large forms into smaller, focused ones.

3. **Debugging**: Use graph visualization to understand the conversation flow and debug issues with field collection.

4. **Export**: You can save graphs as PNG files for documentation or sharing:
   ```python
   visualize_graph(graph, "my_workflow.png")
   ```

5. **Custom LLMs**: You can pass custom LLM instances to control the agent behavior:
   ```python
   from langchain_openai import ChatOpenAI
   llm = ChatOpenAI(model="gpt-4", temperature=0.5)
   graph = get_agent_graph(meta, llm=llm)
   ```

## Conclusion

This notebook demonstrated how to:

1. Extract and visualize LangGraph workflows from Chatfield forms
2. Use different visualization methods (display, save to file, get metadata)
3. Understand how validation rules affect the graph structure  
4. Access graph internals for advanced use cases

The graph visualization helps you understand the conversation flow and can be useful for:
- Documentation
- Debugging conversation logic
- Explaining the system to stakeholders
- Optimizing form design